# 중요 결정사항 및 설명란
deltaprice = (t+1주가) - (t주가) / (t주가) --> 향후 1년 상승측정 
* 1. deltaprice에서 0은 상폐년도인 것 같음.. 확인 필요 왜냐면 주가 있다가 내년에 사라진거니깐 
* 2. deltaprice에서 99는 deltaprice연산때 Null값 나온거임 해당년도에 상장되어 있지 않기때문인 듯
* 3. Consensus 0(의견없음)이 존나 많음, 그리고 hold랑 sell이 전체년도에서 100개 수준인 것 같음
* 4. 처음에 Cut20NullfillNA에서 쓸모없는 데이터 버리는데, 여기서 기본 PEIS데이터만 있을때 20개 NULL값 있는거만 쓸모없었고 나머지는 0 채워서 쓰면댈 듯 근데 은행들 이런친구들은 아에안쓰는 항목들은 PEIS지표에 어떻게 영향을 줄진 모르겠음
* 5. 2010년 2021년 데이터는 없어도 댐 쓰면도 안댐

###     나중에 kospi, kosdaq 별로 따로 읽어서 처리 행야함 + 내 생각에 다중회귀분석은 따로 만들어야할 듯 

# 필요 모듈

In [93]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

from Fin import *
warnings.filterwarnings(action='ignore')

# 0. 엑셀읽기

In [107]:
df = pd.read_excel("Data/KospiPEISData&PriceData.xlsx")
df = Cut20NullfillNA(df)
# 이게 목표주가랑 수정주가 붙ㅇ티면서 NA가 더 많아질것 같은데 그러면 그때는 그냥 20이상은 제거하는 것으로하자
# 여기서 deltaprice이상한거 제거하기
# 여기서부터 년도 별로 나누기 해야할 가능성 있음

# Drop 시킬 데이터 코드모음 --> 나중에 deltaprice해야함 

In [95]:
df = df.drop(df[ (df['deltaprice']==99) | (df['deltaprice']==np.inf)].index)

In [101]:
df = pd.read_excel("Data/KospiPEISData&PriceData.xlsx")

0         0
1         0
2         0
3         0
4         0
         ..
13987    24
13988    24
13989    24
13990    24
13991    24
Length: 13992, dtype: int64

# 1. PEIS를 구하기 위한 기초데이터들 연산해보기 (기업별로)

In [96]:
PEISBaseData = CompuMoreData(df)
PEISBaseData = setSGA(PEISBaseData)

In [97]:
PEISBaseData['']

,Name,회계년,Symbol,총자산(천원),총자산(평균)(천원),영업이익(천원),매출원가(천원),판매비와관리비(천원),금융원가(비영업)(천원),기타비용(비영업)(천원),...,SGA,deltaSGA,GM,deltaGM1,deltasale,deltaGM2,ATO,deltaATO,SGAdeltasale>0,SGAdeltasale<0
0,휴비스,2012,A079980,8.254603e+08,8.095595e+08,58819263.08,1.414522e+09,8.917806e+07,7037479.0,11759372.0,...,1.503700e+09,0.000000e+00,1.479973e+08,0.000000,0.000000,0.000000,1.892907,0.000000,0.00,0.000000e+00
1,휴비스,2013,A079980,8.262735e+08,8.258669e+08,43055543.08,1.399278e+09,9.058445e+07,5038770.0,16131809.0,...,1.489862e+09,-1.383792e+07,1.336400e+08,-0.097011,-0.018945,-0.078066,1.855218,-0.037688,0.00,-1.383792e+07
2,휴비스,2014,A079980,8.996906e+08,8.629820e+08,43517637.97,1.221072e+09,1.071059e+08,6552552.0,16850231.0,...,1.328178e+09,-1.616840e+08,1.506236e+08,0.127084,-0.105173,0.232258,1.524631,-0.330588,0.00,-1.616840e+08
3,휴비스,2015,A079980,8.565891e+08,8.781398e+08,33597427.19,1.069096e+09,1.023916e+08,10152125.0,16151979.0,...,1.171487e+09,-1.566910e+08,1.359891e+08,-0.097159,-0.121464,0.024304,1.406841,-0.117790,0.00,-1.566910e+08
4,휴비스,2016,A079980,9.141253e+08,8.853572e+08,15073251.20,1.027344e+09,1.010837e+08,8065731.0,14554763.0,...,1.128428e+09,-4.305932e+07,1.161570e+08,-0.145836,-0.051103,-0.094733,1.250924,-0.155917,0.00,-4.305932e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8610,부산가스,2016,A015350,8.406410e+08,8.224005e+08,35905166.14,7.550458e+08,9.593077e+07,2415125.0,7109890.0,...,8.509766e+08,-1.722021e+08,1.318359e+08,0.026106,-0.158702,0.184808,1.055007,-0.255905,0.00,-1.722021e+08
8611,부산가스,2017,A015350,9.085391e+08,8.745901e+08,37762515.46,7.980412e+08,9.943127e+07,240462.0,4560750.0,...,8.974725e+08,4.649588e+07,1.371938e+08,0.040640,0.054520,-0.013880,1.029383,-0.025623,46495881.79,0.000000e+00
8612,부산가스,2018,A015350,9.783107e+08,9.434249e+08,41259978.97,8.070351e+08,1.035129e+08,165666.0,3606421.0,...,9.105480e+08,1.307548e+07,1.447729e+08,0.055244,0.017721,0.037523,0.972910,-0.056474,13075480.93,0.000000e+00
8613,부산가스,2019,A015350,1.036741e+09,1.007526e+09,43528727.47,8.186778e+08,1.105826e+08,227343.0,5602763.0,...,9.292604e+08,1.871239e+07,1.541113e+08,0.064504,0.022043,0.042461,0.938315,-0.034595,18712394.20,0.000000e+00


# 매출액delta, 이상치 deltaGNOA문제 --> 둘다 inf있노;
1. inf의 경우 전년도가 매출액 0 이였음 --> 걍뺴자 inf
2, 금융권 친구들이 GNOA 에서 에러가 남 --> 금융권 비금융권 따로 계산해가지고 

In [43]:
deltasale_err_index = PEISBaseData[PEISBaseData['deltasale'] == np.inf].index
PEISBaseData.drop(deltasale_err_index, inplace = True)

In [98]:
PEISBaseData['deltaGNOA'].describe()

count    7984.000000
mean             inf
std              NaN
min      -764.560921
25%        -0.109396
50%         0.000000
75%         0.137502
max              inf
Name: deltaGNOA, dtype: float64

In [49]:
PEISBaseData[PEISBaseData['deltasale'] < -1][['매출액(천원)','deltasale']]

,매출액(천원),deltasale
1166,-7.538471e+06,-1.042741
1167,2.764051e+08,-37.665939
3479,-9.397676e+06,-1.940680
3480,1.716586e+07,-2.826607
3684,7.952744e+07,-1.508919


In [61]:
PEISBaseData.count()

Name                8613
회계년                 8613
Symbol              8613
총자산(천원)             8613
총자산(평균)(천원)         8613
영업이익(천원)            8613
매출원가(천원)            8613
판매비와관리비(천원)         8613
금융원가(비영업)(천원)       8613
기타비용(비영업)(천원)       8613
보통주자본금(천원)          8613
유동부채(천원)            8613
비유동부채(천원)           8613
우선주자본금(천원)          8613
현금및현금성자산(천원)        8613
단기사채(천원)            8613
단기차입금(천원)           8613
매출액(천원)             8613
매출총이익(천원)           8613
영업활동으로인한현금흐름(천원)    8613
매출채권(천원)            8613
재고자산(천원)            8613
매입채무(천원)            8613
ADJprice            8613
Consensus           8613
deltaprice          8613
consen_cut          8613
CountNull           8613
NetAsset            8613
deltaACC            8613
RNOA                8613
GNOA                8613
deltaGNOA           7982
SGA                 8613
deltaSGA            8613
GM                  8613
deltaGM1            8603
deltasale           8601
deltaGM2            8601
ATO                 8613


In [67]:
PEISBaseData['deltaGNOA'].isnull().sum()

631

In [68]:
PEISBaseData['deltasale'].isnull().sum()

12

In [69]:
PEISBaseData['deltaGM2'].isnull().sum()

12

In [72]:
PEISBaseData[PEISBaseData['deltaGNOA'].isnull()][['매출채권(천원)','재고자산(천원)','매입채무(천원)']]

,매출채권(천원),재고자산(천원),매입채무(천원)
96,0.0,0.0,0.0
97,0.0,0.0,0.0
98,0.0,0.0,0.0
99,0.0,0.0,0.0
175,0.0,0.0,0.0
...,...,...,...
8535,0.0,0.0,0.0
8536,0.0,0.0,0.0
8537,0.0,0.0,0.0
8538,0.0,0.0,0.0


In [74]:
PEISBaseData[PEISBaseData['deltaGNOA'].isnull()]['Name'].unique()

array(['한국자산신탁', '키움증권', '신한지주', '하나금융지주', '오렌지라이프', '우리금융지주', '상상인증권',
       'KB손해보험', '한화투자증권', '우리금융캐피탈', '제일저축은행', '미래에셋증권', '우리스팩1호',
       '부산은행', '대신증권', '동양생명', '한화생명', '외환은행', '신영증권', '메리츠증권', '삼성생명',
       '메리츠금융지주', 'NH농협증권', '화인자산관리', '우리종금', '산은캐피탈', 'JB금융지주',
       '솔로몬저축은행', '유화증권', '현대차증권', '한국금융지주', '교보증권', '메리츠화재', 'SBI모기지',
       'DB금융투자', '한국토지신탁', '삼성화재', '흥국화재', '롯데손해보험', '기업은행', '부국증권',
       'NH투자증권', '미래에셋생명', '한미은행', '우리금융', '현대해상', '한국저축은행', 'DB손해보험',
       '대구은행', '한양증권', 'LG카드', '굿모닝신한증권', '그린손해보험', '삼성증권', '제주은행',
       '전북은행', '한화손해보험', 'SK증권', '우리은행', '진흥저축은행', '국민은행', '삼성카드',
       '유안타증권', '대우증권스팩', 'BNK금융지주', '유진투자증권', '현대증권', 'DGB금융지주', 'KB금융',
       '광주은행', '경남은행', '코리안리', '서울저축은행', 'KTB투자증권', '전기초자', '동양밸류스팩',
       'KB캐피탈'], dtype=object)

In [ ]:
PEISBaseData[PEISBaseData['deltaGNOA'].isnull()]['']

In [78]:
PEISBaseData[PEISBaseData['deltasale'].isnull()]['매출액(천원)']

1811    0.0
7172    0.0
8148    0.0
8149    0.0
8150    0.0
8151    0.0
8152    0.0
8153    0.0
8154    0.0
8155    0.0
8272    0.0
8273    0.0
Name: 매출액(천원), dtype: float64

In [75]:
PEISBaseData[PEISBaseData['deltasale'].isnull()][['매출액(천원)','deltasale']]

,매출액(천원),deltasale
1811,0.0,NaN
7172,0.0,NaN
8148,0.0,NaN
8149,0.0,NaN
8150,0.0,NaN
8151,0.0,NaN
8152,0.0,NaN
8153,0.0,NaN
8154,0.0,NaN
8155,0.0,NaN


In [130]:
PEISBaseData.loc[7171:7173,['회계년','매출액(천원)','deltasale']]

,회계년,매출액(천원),deltasale
7171,2010,0.000000e+00,0.0
7172,2011,0.000000e+00,NaN
7173,2010,7.997521e+08,0.0


In [129]:
len(PEISBaseData['Name'].unique())

891

In [119]:
PEISBaseData[PEISBaseData['deltasale'] == 0][['매출액(천원)','deltasale']]

,매출액(천원),deltasale
0,1.562519e+09,0.0
9,2.198912e+09,0.0
20,5.124275e+09,0.0
31,1.417417e+08,0.0
42,3.941219e+08,0.0
...,...,...
8562,6.463960e+07,0.0
8571,1.033055e+09,0.0
8582,4.432757e+08,0.0
8593,2.536586e+08,0.0


# 2. 가공된 데이터 토대로 +1,-1,0 구하기 (년도별로)

In [79]:
Yearlydfs = yearbaseDFs(PEISBaseData, years =[2011,2012,2013,2014,2015,2016,2017,2018,2019,2020])
PEISBaseData = CalPEISpoints(Yearlydfs)

# 3. PEIS 점수 구하기 (단순 행 sum)

In [80]:
PEISBaseData = CalPEIS(PEISBaseData)

# 4. 년도 별 포트폴리오 구하기 위해 년도 별로 나누기

In [81]:
dfs = yearbaseDFs(PEISBaseData, years = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020])

## 5. 조건 & 년도별로 포트폴리오 수익률 구하기 (equally weight)

In [10]:
PEISbuyport = Upper20Port(dfs,"valPEIS")
np.mean(PEISbuyport)

0.12086963609933053

# 6. 엑셀로 보내기 

In [11]:
writer = pd.ExcelWriter('Result/Compute_PEIS.xlsx', engine='xlsxwriter')

In [12]:
# 이거 나중에 수정 필요
collist=['ADJprice', 'Consensus',\
       'deltaprice', 'CountNull', 'NetAsset', 'deltaACC', 'RNOA', 'GNOA',\
       'deltaGNOA', 'SGA', 'deltaSGA', 'GM', 'deltaGM1', 'deltasale',\
       'deltaGM2', 'ATO', 'deltaATO', 'SGAdeltasale>0', 'SGAdeltasale<0',\
       'GNOAPEISpoint', 'deltaATOPEISpoint', 'deltaGM2PEISpoint',\
       'deltaACCPEISpoint', 'RNOAPEISpoint', 'SGAdeltasale>0PEISpoint',\
       'SGAdeltasale<0PEISpoint', 'valPEIS']

In [13]:
Computed = PEISBaseData[collist]

In [14]:
PEISBaseData.to_excel(writer,sheet_name = 'All_data')
Computed.to_excel(writer,sheet_name = 'Computed')

In [15]:
writer.save()

# 이 아래에서는 이것저것 실험